In [1]:
import numpy as np
import tensorflow as tf
import os
import tqdm
import glob
import sys
import matplotlib.pyplot as plt
import pickle
import multiprocessing
import itertools
import random
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
with open("train_files.pickle", "rb") as f:
    files_train = pickle.load(f)
with open("val_files.pickle", "rb") as f:
    files_val = pickle.load(f)

In [4]:
labels_train = []
phonemes_train = []
mfcc_train = []
video_train = []
for file in tqdm.tqdm(files_train):
    cur_labels = np.load(file)["labels"]
    cur_phonemes = np.load(file)["phonemes"]
    cur_mfcc = np.load(file)["mfcc"]
    cur_mfcc = (cur_mfcc - cur_mfcc.mean(axis=0))/cur_mfcc.std(axis=0)
    cur_video = np.load(file)["video"]
    labels_train.append(cur_labels)
    phonemes_train.append(cur_phonemes)
    mfcc_train.append(cur_mfcc)
    video_train.append(cur_video)

100%|██████████| 8433/8433 [01:05<00:00, 128.64it/s]


In [5]:
labels_val = []
phonemes_val = []
mfcc_val = []
video_val = []
for file in tqdm.tqdm(files_val):
    cur_labels = np.load(file)["labels"]
    cur_phonemes = np.load(file)["phonemes"]
    cur_mfcc = np.load(file)["mfcc"]
    cur_mfcc = (cur_mfcc - cur_mfcc.mean(axis=0))/cur_mfcc.std(axis=0)
    cur_video = np.load(file)["video"]
    labels_val.append(cur_labels)
    phonemes_val.append(cur_phonemes)
    mfcc_val.append(cur_mfcc)
    video_val.append(cur_video)

100%|██████████| 937/937 [00:07<00:00, 133.20it/s]


In [6]:
video_val[0].shape

(529, 30, 50)

In [7]:
def conv_layer(prev_layer, filter_width, num_filters, use_relu=True, use_batchnorn=True, is_training=None):
    convolution_out = tf.layers.conv1d(prev_layer, num_filters, filter_width, strides=1, padding="same", 
                                  activation=None)
    if use_batchnorn:
        if is_training is None:
            raise Exception("is_training placeholder required")
        convolution_out = tf.layers.batch_normalization(convolution_out, training=is_training)
    if use_relu:
        convolution_out = tf.nn.relu(convolution_out)
    return convolution_out

In [34]:
class PhonemeDNNModel:
    def __init__(self, num_features=(30, 50), num_symbols=23, optimizer="adam", use_batchnorm=True, folder="dnn_mfcc"):
        tf.reset_default_graph()
        self.num_features = num_features
        self.num_symbols = num_symbols
        self.checkpoints_folder = folder
        self.epoch = 0
        self.step = 0
        self.min_dev_loss = np.float('inf')
        self.global_step = tf.Variable(0, trainable=False, name="global_step")
        self.optimizer_type = optimizer
        self.use_batchnorm = use_batchnorm
        self._build_graph()

    def _create_placeholders(self):
        self.features_placeholder = tf.placeholder(tf.float32, [None, None, 30, 50, 1], name="features")
        self.features_len_placeholder = tf.placeholder(tf.int32, [None], name="features_len")
        self.target_placeholder = tf.placeholder(tf.int32, [None, None], name="targets")
        self.target_len_placeholder = tf.placeholder(tf.int32, [None], name="targets_len")

        self.learning_rate = tf.placeholder_with_default(1e-4, [], name="learning_rate")
        self.is_training = tf.placeholder_with_default(False, [], name="is_training")

        # transform target to sparse
#         target_ind = tf.where(tf.not_equal(tf.sequence_mask(self.target_len_placeholder), False))
#         target_val = tf.gather_nd(self.target_placeholder, target_ind)
#         self.target_sparse = tf.SparseTensor(target_ind, target_val,
#                                              tf.cast(tf.shape(self.target_placeholder), dtype=tf.int64))

    def _inference(self):
        # define architecture
        self.dropout = tf.cond(self.is_training, lambda: tf.constant(0.5), lambda: tf.constant(1.0))
        outputs = self.features_placeholder
        outputs = tf.layers.conv3d(outputs, filters=32, kernel_size=(3, 5, 5), activation=tf.nn.relu, padding="same")
        outputs = tf.layers.max_pooling3d(outputs, (1, 2, 2), (1, 2, 2), padding="same")
#         outputs = tf.nn.dropout(outputs, self.dropout)
        
        outputs = tf.layers.conv3d(outputs, filters=64, kernel_size=(3, 5, 5), activation=tf.nn.relu, padding="same")
        outputs = tf.layers.max_pooling3d(outputs, (1, 2, 2), (1, 2, 2), padding="same")
#         outputs = tf.nn.dropout(outputs, self.dropout)
        
        outputs = tf.layers.conv3d(outputs, filters=96, kernel_size=(3, 3, 3), activation=tf.nn.relu, padding="same")
        outputs = tf.layers.max_pooling3d(outputs, (1, 2, 2), (1, 2, 2), padding="same")
#         outputs = tf.nn.dropout(outputs, self.dropout)
        
        shape = tf.shape(outputs)
#         print(outputs)
        outputs = tf.reshape(outputs, (shape[0], shape[1], 4 * 7 * 96))
        
        with tf.variable_scope("rnn1"):
            cell1 = tf.nn.rnn_cell.GRUCell(256)
            outputs1, _ = tf.nn.bidirectional_dynamic_rnn(cell1, cell1, outputs, sequence_length=self.features_len_placeholder, dtype=tf.float32)
            outputs = tf.concat(outputs1, axis=-1)

        with tf.variable_scope("rnn2"):
            cell2 = tf.nn.rnn_cell.GRUCell(256)
            outputs2, _ = tf.nn.bidirectional_dynamic_rnn(cell2, 
                                            cell2, outputs, sequence_length=self.features_len_placeholder, dtype=tf.float32)
            outputs = tf.concat(outputs2, axis=-1)
        outputs = tf.layers.dense(outputs, self.num_symbols)
        logits_batch_major = outputs

        self.logits = logits_batch_major  # tf.transpose(logits_batch_major, [1, 0, 2])


    def _create_loss(self):
        self.predictions = tf.argmax(self.logits, axis=-1)
        self.predictions_prob = tf.nn.softmax(self.logits)
        mask = tf.sequence_mask(self.target_len_placeholder, dtype=tf.float32)
        self.accuracy = tf.reduce_sum(tf.cast(
            tf.equal(tf.cast(self.predictions, tf.int32), 
            self.target_placeholder), tf.float32) * mask) / tf.cast(tf.reduce_sum(self.target_len_placeholder), tf.float32)
        with tf.name_scope('training'):
            cost = tf.contrib.seq2seq.sequence_loss(self.logits, self.target_placeholder, mask)
            self.loss = tf.reduce_mean(cost, name='average_loss')


    def _create_optimizer(self):
        if self.optimizer_type == "adam":
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
        elif self.optimizer_type == "yellowfin":
            raise ValueError("not now")
        else:
            raise ValueError("incorrect optimizer")

        gvs = self.optimizer.compute_gradients(self.loss)
        gradients, trainables = zip(*gvs)
        clipped_gradients, norm = tf.clip_by_global_norm(gradients, 5.0, name='clip_gradients')
        # batch normalization in tensorflow requires this extra dependency
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(extra_update_ops):
            self.train_step = self.optimizer.apply_gradients(zip(clipped_gradients, trainables),
                                                             global_step=self.global_step)


    def _build_graph(self):
        self._create_placeholders()
        self._inference()
        self._create_loss()
        self._network_vars = tf.global_variables()  # without optimizer
        self._create_optimizer()
        self.saver = tf.train.Saver()
        # self._create_summary()


    def save_weights(self, sess):
        self.saver.save(sess, 
            '/home/artbataev/Documents/checkpoints/{}/adam_checkpoints_best/ckpt'.format(self.checkpoints_folder), 
            self.step)


    def restore_weights(self, sess, only_network=False):
        ckpt = tf.train.get_checkpoint_state(
            '/home/artbataev/Documents/checkpoints/{}/adam_checkpoints_best/'.format(self.checkpoints_folder))
        if ckpt and ckpt.model_checkpoint_path:
            if only_network:
                saver = tf.train.Saver(self._network_vars)
                saver.restore(sess, ckpt.model_checkpoint_path)
            else:
                self.saver.restore(sess, ckpt.model_checkpoint_path)


    def init_op(self, sess, restore=True, restore_only_network=False):
        if restore:
            sess.run(tf.global_variables_initializer())
            self.restore_weights(sess, only_network=restore_only_network)
        else:
            sess.run(tf.global_variables_initializer())
    
    def get_predictions_p(self, features, features_len):
        feed_dict = {
                self.features_placeholder: features.reshape(1, features.shape[0], features.shape[1], features.shape[2], 1),
                self.features_len_placeholder: features_len}
        pred_p = sess.run(self.predictions_prob, feed_dict=feed_dict)
        return pred_p[0]
    
    def get_features_video(self, features, features_len):
        features_tensor = tf.get_default_graph().get_tensor_by_name("rnn2/concat:0")
        feed_dict = {
                self.features_placeholder: features.reshape(1, features.shape[0], features.shape[1], features.shape[2], 1),
                self.features_len_placeholder: features_len}
        pred_p = sess.run(features_tensor, feed_dict=feed_dict)
        return pred_p[0]
    
    def test_net(self, batch_generator, sess, verbose=False):
        last = False
        full_len = 0
        loss = 0.0
        accuracy = 0.0
        while not last:
            texts, texts_len, features, features_len, last = next(batch_generator)
            feed_dict = {
                self.features_placeholder: features,
                self.features_len_placeholder: features_len,
                self.target_placeholder: texts,
                self.target_len_placeholder: texts_len
            }

            current_loss, current_acc = sess.run([self.loss, self.accuracy], feed_dict=feed_dict)

            loss += current_loss * len(texts)
            accuracy += current_acc * len(texts)
            full_len += len(texts)
        accuracy /= full_len
        loss /= full_len
        if verbose:
            print("Average test loss: {:.5f}".format(loss))
            print("Average test Accuracy: {:.2f}%".format(accuracy * 100))
        return loss, accuracy


    def run_step(self, batch_generator, sess, lr=1e-4):
        texts, texts_len, features, features_len, last = next(batch_generator)
#         print(features_len, texts_len)
        feed_dict = {
            self.features_placeholder: features,
            self.features_len_placeholder: features_len,
            self.target_placeholder: texts,
            self.target_len_placeholder: texts_len,
            self.is_training: True,
            self.learning_rate: lr,
        }

        current_loss, current_acc, _ = sess.run([self.loss, self.accuracy, self.train_step], feed_dict=feed_dict)
        self.step += 1
        if last:
            self.epoch += 1
            print("Epoch {}, end of dataset".format(self.epoch))
        return current_loss, current_acc


    def fit(self, batch_generator, dev_generator, sess, steps=1, lr=1e-4):
        try:
            train_loss = 0.0
            train_acc = 0.0
            for _ in range(steps):
                current_loss, current_acc = self.run_step(batch_generator, sess, lr)
                train_loss += current_loss
                train_acc += current_acc

                if self.step % 10 == 0:
                    train_loss /= 10
                    train_acc /= 10
                    print("Epoch {} step {} average_loss: {:.5f} Acc: {:.2f}%".format(
                        self.epoch, self.step, train_loss, train_acc * 100))
                else:
                    print(".", end="")

                if self.step % 50 == 0:
                    epoch_test_loss, epoch_test_acc = self.test_net(dev_generator, sess)
                    improved_text = "improved" if epoch_test_loss < self.min_dev_loss else "not improved"
                    print("Step {} test_loss: {:.5f} Acc: {:.2f}%".format(self.step, epoch_test_loss,
                                                                          np.mean(epoch_test_acc) * 100), improved_text)
                    if epoch_test_loss < self.min_dev_loss:
                        self.min_dev_loss = epoch_test_loss
                        self.save_weights(sess)

                if self.step % 10 == 0:
                    train_loss = 0.0
                    train_acc = 0.0


        except KeyboardInterrupt:
            print("Training Interrupted")

In [9]:
batch_size = 4

In [10]:
def train_generator():
    global batch_size
    while True:
        all_indices = np.random.choice(len(labels_train), len(labels_train), replace=False)
        for j in range(0, len(labels_train), batch_size):
            indices = all_indices[j: j + batch_size]
            texts_len = np.array(list(map(lambda x: len(x), (phonemes_train[i] for i in indices))))
            features_len = np.array(list(map(lambda x: len(x), (video_train[i] for i in indices))))
            texts = np.zeros((texts_len.shape[0], np.max(texts_len)), dtype=np.int)
            features = np.zeros((texts_len.shape[0], np.max(features_len), 30, 50))
            #         texts, texts_len, mfcc, mfcc_len, last
            for i in range(texts_len.shape[0]):
                texts[i, :len(phonemes_train[indices[i]])] = phonemes_train[indices[i]]
                features[i, :len(video_train[indices[i]])] = video_train[indices[i]]
            features = features[..., np.newaxis]
            yield texts, texts_len, features, features_len, j + batch_size >= len(labels_train)

In [11]:
def val_generator():
    batch_size = 8
#     global batch_size
    while True:
        for j in range(0, len(phonemes_val), batch_size):
            indices = np.arange(j, min(j + batch_size, len(phonemes_val)))
            texts_len = np.array(list(map(lambda x: len(x), (phonemes_val[i] for i in indices))))
            features_len = np.array(list(map(lambda x: len(x), (video_val[i] for i in indices))))
            texts = np.zeros((texts_len.shape[0], np.max(texts_len)), dtype=np.int)
            features = np.zeros((texts_len.shape[0], np.max(features_len), 30, 50))
            #         texts, texts_len, mfcc, mfcc_len, last
            for i in range(texts_len.shape[0]):
                texts[i, :len(phonemes_val[indices[i]])] = phonemes_val[indices[i]]
                features[i, :len(video_val[indices[i]])] = video_val[indices[i]]
            features = features[..., np.newaxis]
            yield texts, texts_len, features, features_len, (j + batch_size >= len(phonemes_val))

In [12]:
tgen = train_generator()
vgen = val_generator()

In [263]:
dnn = PhonemeDNNModel(optimizer="adam", use_batchnorm=False, folder="dnn_video")
sess = tf.Session()
dnn.init_op(sess, restore=False)

In [264]:
# dnn.run_step(tgen, sess)

In [ ]:
dnn.fit(tgen, vgen, sess, steps=1000)

.........Epoch 0 step 10 average_loss: 2.86819 Acc: 27.15%
.........Epoch 0 step 20 average_loss: 2.81652 Acc: 26.67%
.........Epoch 0 step 30 average_loss: 2.69050 Acc: 31.55%
.........Epoch 0 step 40 average_loss: 2.74946 Acc: 29.05%
.........Epoch 0 step 50 average_loss: 2.74001 Acc: 29.15%
Step 50 test_loss: 2.71822 Acc: 29.49% improved
.........Epoch 0 step 60 average_loss: 2.70765 Acc: 30.43%
.........Epoch 0 step 70 average_loss: 2.62562 Acc: 31.66%
.........Epoch 0 step 80 average_loss: 2.63696 Acc: 31.30%
.........Epoch 0 step 90 average_loss: 2.53085 Acc: 32.66%
.........Epoch 0 step 100 average_loss: 2.67588 Acc: 27.13%
Step 100 test_loss: 2.54257 Acc: 30.31% improved
.........Epoch 0 step 110 average_loss: 2.41080 Acc: 35.45%
.........Epoch 0 step 120 average_loss: 2.63856 Acc: 25.90%
.........Epoch 0 step 130 average_loss: 2.54365 Acc: 31.23%
.........Epoch 0 step 140 average_loss: 2.51406 Acc: 30.85%
.........Epoch 0 step 150 average_loss: 2.32929 Acc: 35.42%
Step 150 tes

In [ ]:
dnn.fit(tgen, vgen, sess, steps=100000)

.........Epoch 0 step 1010 average_loss: 1.55566 Acc: 51.70%
.........Epoch 0 step 1020 average_loss: 1.55809 Acc: 51.79%
.........Epoch 0 step 1030 average_loss: 1.53858 Acc: 52.27%
.........Epoch 0 step 1040 average_loss: 1.52984 Acc: 52.31%
.........Epoch 0 step 1050 average_loss: 1.59668 Acc: 51.20%
Step 1050 test_loss: 1.62445 Acc: 49.76% improved
.........Epoch 0 step 1060 average_loss: 1.62687 Acc: 49.53%
.........Epoch 0 step 1070 average_loss: 1.54612 Acc: 51.62%
.........Epoch 0 step 1080 average_loss: 1.67270 Acc: 50.79%
.........Epoch 0 step 1090 average_loss: 1.64455 Acc: 49.53%
.........Epoch 0 step 1100 average_loss: 1.46026 Acc: 55.67%
Step 1100 test_loss: 1.59584 Acc: 50.92% improved
.........Epoch 0 step 1110 average_loss: 1.66589 Acc: 48.56%
.........Epoch 0 step 1120 average_loss: 1.63566 Acc: 50.16%
.........Epoch 0 step 1130 average_loss: 1.60190 Acc: 52.45%
.........Epoch 0 step 1140 average_loss: 1.42685 Acc: 54.05%
.........Epoch 0 step 1150 average_loss: 1.459

.........Epoch 1 step 2160 average_loss: 1.43388 Acc: 54.23%
.........Epoch 1 step 2170 average_loss: 1.33837 Acc: 58.05%
.........Epoch 1 step 2180 average_loss: 1.29124 Acc: 58.21%
.........Epoch 1 step 2190 average_loss: 1.32406 Acc: 58.37%
.........Epoch 1 step 2200 average_loss: 1.30706 Acc: 58.34%
Step 2200 test_loss: 1.28153 Acc: 58.89% improved
.........Epoch 1 step 2210 average_loss: 1.35915 Acc: 56.81%
.........Epoch 1 step 2220 average_loss: 1.35098 Acc: 58.77%
.........Epoch 1 step 2230 average_loss: 1.50962 Acc: 54.64%
.........Epoch 1 step 2240 average_loss: 1.21841 Acc: 61.32%
.........Epoch 1 step 2250 average_loss: 1.44165 Acc: 55.29%
Step 2250 test_loss: 1.28435 Acc: 59.23% not improved
.........Epoch 1 step 2260 average_loss: 1.46200 Acc: 52.89%
.........Epoch 1 step 2270 average_loss: 1.26327 Acc: 59.32%
.........Epoch 1 step 2280 average_loss: 1.17489 Acc: 62.67%
.........Epoch 1 step 2290 average_loss: 1.19353 Acc: 59.85%
.........Epoch 1 step 2300 average_loss: 1

.........Epoch 1 step 3310 average_loss: 1.05101 Acc: 64.91%
.........Epoch 1 step 3320 average_loss: 1.14787 Acc: 61.23%
.........Epoch 1 step 3330 average_loss: 1.10454 Acc: 64.43%
.........Epoch 1 step 3340 average_loss: 1.19129 Acc: 61.56%
.........Epoch 1 step 3350 average_loss: 1.21727 Acc: 59.70%
Step 3350 test_loss: 1.12144 Acc: 62.83% improved
.........Epoch 1 step 3360 average_loss: 0.99270 Acc: 66.86%
.........Epoch 1 step 3370 average_loss: 1.27880 Acc: 58.40%
.........Epoch 1 step 3380 average_loss: 1.19187 Acc: 60.16%
.........Epoch 1 step 3390 average_loss: 1.23612 Acc: 61.12%
.........Epoch 1 step 3400 average_loss: 1.19177 Acc: 60.45%
Step 3400 test_loss: 1.13259 Acc: 62.37% not improved
.........Epoch 1 step 3410 average_loss: 1.10940 Acc: 61.81%
.........Epoch 1 step 3420 average_loss: 1.11884 Acc: 62.73%
.........Epoch 1 step 3430 average_loss: 1.06286 Acc: 63.18%
.........Epoch 1 step 3440 average_loss: 1.29765 Acc: 58.44%
.........Epoch 1 step 3450 average_loss: 1

.........Epoch 2 step 4460 average_loss: 1.09101 Acc: 62.66%
.........Epoch 2 step 4470 average_loss: 1.03900 Acc: 64.42%
.........Epoch 2 step 4480 average_loss: 0.97064 Acc: 67.56%
.........Epoch 2 step 4490 average_loss: 1.10688 Acc: 63.24%
.........Epoch 2 step 4500 average_loss: 0.88232 Acc: 69.32%
Step 4500 test_loss: 1.04545 Acc: 64.85% not improved
.........Epoch 2 step 4510 average_loss: 1.08820 Acc: 63.24%
.........Epoch 2 step 4520 average_loss: 0.90738 Acc: 69.15%
.........Epoch 2 step 4530 average_loss: 1.04000 Acc: 64.82%
.........Epoch 2 step 4540 average_loss: 1.22292 Acc: 58.12%
.........Epoch 2 step 4550 average_loss: 1.02038 Acc: 65.58%
Step 4550 test_loss: 1.02480 Acc: 65.08% not improved
.........Epoch 2 step 4560 average_loss: 1.04106 Acc: 65.22%
.........Epoch 2 step 4570 average_loss: 1.11303 Acc: 62.68%
.........Epoch 2 step 4580 average_loss: 1.06906 Acc: 64.99%
.........Epoch 2 step 4590 average_loss: 1.02811 Acc: 63.81%
.........Epoch 2 step 4600 average_los

.........Epoch 2 step 5610 average_loss: 0.97970 Acc: 67.30%
.........Epoch 2 step 5620 average_loss: 0.87862 Acc: 68.54%
.........Epoch 2 step 5630 average_loss: 1.23373 Acc: 62.00%
.........Epoch 2 step 5640 average_loss: 0.93600 Acc: 67.91%
.........Epoch 2 step 5650 average_loss: 0.82428 Acc: 71.29%
Step 5650 test_loss: 0.98857 Acc: 65.67% not improved
.........Epoch 2 step 5660 average_loss: 0.95922 Acc: 66.27%
.........Epoch 2 step 5670 average_loss: 1.05495 Acc: 63.61%
.........Epoch 2 step 5680 average_loss: 0.93802 Acc: 66.28%
.........Epoch 2 step 5690 average_loss: 1.07818 Acc: 63.56%
.........Epoch 2 step 5700 average_loss: 0.93075 Acc: 68.22%
Step 5700 test_loss: 0.96717 Acc: 66.81% not improved
.........Epoch 2 step 5710 average_loss: 1.04592 Acc: 65.29%
.........Epoch 2 step 5720 average_loss: 1.05241 Acc: 63.87%
.........Epoch 2 step 5730 average_loss: 0.88129 Acc: 69.24%
.........Epoch 2 step 5740 average_loss: 0.93837 Acc: 67.69%
.........Epoch 2 step 5750 average_los

Step 6750 test_loss: 0.89482 Acc: 68.31% improved
.........Epoch 3 step 6760 average_loss: 0.77527 Acc: 72.67%
.........Epoch 3 step 6770 average_loss: 0.86265 Acc: 69.75%
.........Epoch 3 step 6780 average_loss: 0.97066 Acc: 67.10%
.........Epoch 3 step 6790 average_loss: 0.83050 Acc: 70.64%
.........Epoch 3 step 6800 average_loss: 0.81154 Acc: 72.39%
Step 6800 test_loss: 0.88473 Acc: 68.73% improved
.........Epoch 3 step 6810 average_loss: 0.93772 Acc: 66.45%
.........Epoch 3 step 6820 average_loss: 0.88905 Acc: 67.96%
.........Epoch 3 step 6830 average_loss: 0.79916 Acc: 70.46%
.........Epoch 3 step 6840 average_loss: 0.85903 Acc: 70.46%
.........Epoch 3 step 6850 average_loss: 0.84667 Acc: 70.62%
Step 6850 test_loss: 0.88651 Acc: 68.73% not improved
.........Epoch 3 step 6860 average_loss: 0.93912 Acc: 67.39%
.........Epoch 3 step 6870 average_loss: 0.77976 Acc: 72.17%
.........Epoch 3 step 6880 average_loss: 0.79478 Acc: 71.77%
.........Epoch 3 step 6890 average_loss: 0.80108 Acc:

.........Epoch 3 step 7940 average_loss: 0.80949 Acc: 71.46%
.........Epoch 3 step 7950 average_loss: 0.83633 Acc: 71.56%
Step 7950 test_loss: 0.85788 Acc: 69.73% not improved
.........Epoch 3 step 7960 average_loss: 0.76127 Acc: 73.22%
.........Epoch 3 step 7970 average_loss: 0.71103 Acc: 74.81%
.........Epoch 3 step 7980 average_loss: 0.93700 Acc: 67.96%
.........Epoch 3 step 7990 average_loss: 0.77367 Acc: 71.69%
.........Epoch 3 step 8000 average_loss: 0.86320 Acc: 71.45%
Step 8000 test_loss: 0.83930 Acc: 70.26% improved
.........Epoch 3 step 8010 average_loss: 0.81657 Acc: 70.61%
.........Epoch 3 step 8020 average_loss: 0.64989 Acc: 76.05%
.........Epoch 3 step 8030 average_loss: 0.82311 Acc: 71.10%
.........Epoch 3 step 8040 average_loss: 0.89015 Acc: 68.66%
.........Epoch 3 step 8050 average_loss: 0.78424 Acc: 73.21%
Step 8050 test_loss: 0.84527 Acc: 70.06% not improved
.........Epoch 3 step 8060 average_loss: 0.89398 Acc: 68.53%
.........Epoch 3 step 8070 average_loss: 0.98715 

.........Epoch 4 step 9080 average_loss: 0.76572 Acc: 73.26%
.........Epoch 4 step 9090 average_loss: 0.94329 Acc: 68.15%
.........Epoch 4 step 9100 average_loss: 0.78087 Acc: 71.85%
Step 9100 test_loss: 0.80742 Acc: 71.30% not improved
.........Epoch 4 step 9110 average_loss: 0.64907 Acc: 76.17%
.........Epoch 4 step 9120 average_loss: 0.75011 Acc: 73.34%
.........Epoch 4 step 9130 average_loss: 0.67959 Acc: 74.88%
.........Epoch 4 step 9140 average_loss: 0.90537 Acc: 68.43%
.........Epoch 4 step 9150 average_loss: 0.85574 Acc: 72.01%
Step 9150 test_loss: 0.79768 Acc: 71.33% improved
.........Epoch 4 step 9160 average_loss: 0.63912 Acc: 75.72%
.........Epoch 4 step 9170 average_loss: 0.82228 Acc: 70.60%
.........Epoch 4 step 9180 average_loss: 0.73762 Acc: 72.22%
.........Epoch 4 step 9190 average_loss: 0.70610 Acc: 73.99%
.........Epoch 4 step 9200 average_loss: 0.76597 Acc: 72.66%
Step 9200 test_loss: 0.80783 Acc: 71.27% not improved
.........Epoch 4 step 9210 average_loss: 0.68900 

In [ ]:
# some more steps till ~12000

In [35]:
dnn = PhonemeDNNModel(optimizer="adam", use_batchnorm=False, folder="dnn_video")
sess = tf.Session()
dnn.init_op(sess, restore=True)

INFO:tensorflow:Restoring parameters from /home/artbataev/Documents/checkpoints/dnn_video/adam_checkpoints_best/ckpt-11200


In [14]:
dnn.test_net(vgen, sess)

(0.73478692006657698, 0.73525814516440013)

In [15]:
files_test = list(glob.glob("../data/lip_reading/synchronized/test/*.npz"))

In [16]:
labels_test = []
mfcc_test = []
video_test = []
for file in tqdm.tqdm(files_test):
    labels_test.append(np.load(file)["labels"])
    cur_mfcc = np.load(file)["mfcc"]
    mfcc_test.append((cur_mfcc - cur_mfcc.mean(axis=0)) / cur_mfcc.std(axis=0))
    video_test.append(np.load(file)["video"])

100%|██████████| 2909/2909 [00:28<00:00, 103.20it/s]


In [17]:
predictions_path = "../data/lip_reading/3_predictions/video_only_dnn/"

In [24]:
for i, file in enumerate(files_test):
    predictions = dnn.get_predictions_p(video_test[i], np.array(len(video_test[i])).reshape(1,))
    np.save(predictions_path + os.path.splitext(os.path.basename(file))[0], predictions.astype(np.float32))
    print("processed: " + file + "(frames num: {})".format(predictions.shape[0]))

processed: ../data/lip_reading/synchronized/test/M0083_03_48790_Android_htc.npz(frames num: 150)
processed: ../data/lip_reading/synchronized/test/F0042_02_17538_Android_htc.npz(frames num: 278)
processed: ../data/lip_reading/synchronized/test/M0053_03_97513_Android_SM.npz(frames num: 299)
processed: ../data/lip_reading/synchronized/test/M0081_02_10694_Android_SM_Gal_J3.npz(frames num: 318)
processed: ../data/lip_reading/synchronized/test/M0047_01_69175_Android_htc.npz(frames num: 310)
processed: ../data/lip_reading/synchronized/test/M0016_02_1763092458_iPhone_Iphone5.npz(frames num: 826)
processed: ../data/lip_reading/synchronized/test/F0022_03_73890_Android_SM.npz(frames num: 414)
processed: ../data/lip_reading/synchronized/test/F0022_01_2637015489_iPhone_6s.npz(frames num: 519)
processed: ../data/lip_reading/synchronized/test/F0130_01_1935482760_Android_SM.npz(frames num: 580)
processed: ../data/lip_reading/synchronized/test/F0052_03_02864_iPhone_6s.npz(frames num: 187)
processed: ..

processed: ../data/lip_reading/synchronized/test/M0037_01_75963_Android_htc.npz(frames num: 318)
processed: ../data/lip_reading/synchronized/test/M0056_01_48150_iPhone_6s.npz(frames num: 232)
processed: ../data/lip_reading/synchronized/test/M0030_03_91605_iPhone_6s.npz(frames num: 349)
processed: ../data/lip_reading/synchronized/test/F0117_01_2571963048_Android_SM.npz(frames num: 548)
processed: ../data/lip_reading/synchronized/test/M0039_02_2174905683_Android_htc.npz(frames num: 678)
processed: ../data/lip_reading/synchronized/test/F0090_01_3019456782_iPhone_6s.npz(frames num: 836)
processed: ../data/lip_reading/synchronized/test/M0057_03_9801763425_iPhone_6s.npz(frames num: 550)
processed: ../data/lip_reading/synchronized/test/M0030_03_2107835946_iPhone_6s.npz(frames num: 536)
processed: ../data/lip_reading/synchronized/test/M0055_03_89462_Android_nexus.npz(frames num: 440)
processed: ../data/lip_reading/synchronized/test/F0034_02_95230_Android_htc.npz(frames num: 314)
processed: ../

processed: ../data/lip_reading/synchronized/test/M0081_01_03716_iPhone_6s.npz(frames num: 262)
processed: ../data/lip_reading/synchronized/test/M0039_03_41628_Android_nexus.npz(frames num: 388)
processed: ../data/lip_reading/synchronized/test/F0181_02_20759_Android_nexus.npz(frames num: 260)
processed: ../data/lip_reading/synchronized/test/F0005_02_4910725836_Android_htc.npz(frames num: 582)
processed: ../data/lip_reading/synchronized/test/F0052_01_5326071948_iPhone_6s.npz(frames num: 784)
processed: ../data/lip_reading/synchronized/test/M0089_03_2738015469_iPhone_iphone6.npz(frames num: 824)
processed: ../data/lip_reading/synchronized/test/M0004_02_2691345807_Android_htc.npz(frames num: 670)
processed: ../data/lip_reading/synchronized/test/F0011_02_62138_Android_htc.npz(frames num: 422)
processed: ../data/lip_reading/synchronized/test/F0020_01_87901_iPhone_6s.npz(frames num: 376)
processed: ../data/lip_reading/synchronized/test/M0070_02_5692438017_Android_SM_Gal_J3.npz(frames num: 824

processed: ../data/lip_reading/synchronized/test/M0053_02_5397410268_iPhone_6s.npz(frames num: 475)
processed: ../data/lip_reading/synchronized/test/M0044_02_04279_iPhone_iphone6.npz(frames num: 359)
processed: ../data/lip_reading/synchronized/test/M0002_03_74126_Android_nexus.npz(frames num: 427)
processed: ../data/lip_reading/synchronized/test/F0050_03_53067_Android_SM.npz(frames num: 440)
processed: ../data/lip_reading/synchronized/test/F0182_01_6187249305_Android_SM.npz(frames num: 587)
processed: ../data/lip_reading/synchronized/test/M0013_03_45921_Android_SM_Gal_J3.npz(frames num: 382)
processed: ../data/lip_reading/synchronized/test/M0083_02_03271_Android_nexus.npz(frames num: 222)
processed: ../data/lip_reading/synchronized/test/M0026_01_48150_iPhone_6s.npz(frames num: 301)
processed: ../data/lip_reading/synchronized/test/F0021_02_1392408567_iPhone_iphone6.npz(frames num: 488)
processed: ../data/lip_reading/synchronized/test/M0065_03_9075642813_Android_htc.npz(frames num: 562)


processed: ../data/lip_reading/synchronized/test/F0182_02_0369245817_Android_nexus.npz(frames num: 612)
processed: ../data/lip_reading/synchronized/test/M0071_03_60148_Android_htc.npz(frames num: 294)
processed: ../data/lip_reading/synchronized/test/F0051_02_62138_Android_htc.npz(frames num: 342)
processed: ../data/lip_reading/synchronized/test/M0016_02_6714258309_Android_htc.npz(frames num: 750)
processed: ../data/lip_reading/synchronized/test/F0169_03_24706_Android_nexus.npz(frames num: 484)
processed: ../data/lip_reading/synchronized/test/M0089_02_1268057439_Android_SM_Gal_J3.npz(frames num: 856)
processed: ../data/lip_reading/synchronized/test/M0045_01_4802765319_Android_htc.npz(frames num: 878)
processed: ../data/lip_reading/synchronized/test/M0039_02_29845_Android_htc.npz(frames num: 490)
processed: ../data/lip_reading/synchronized/test/M0033_02_1392408567_iPhone_iphone6.npz(frames num: 641)
processed: ../data/lip_reading/synchronized/test/F0154_03_9263507184_Android_nexus.npz(fr

...

processed: ../data/lip_reading/synchronized/test/M0013_01_1324670895_iPhone_6s.npz(frames num: 798)
processed: ../data/lip_reading/synchronized/test/F0069_01_2593068741_iPhone_6s.npz(frames num: 585)
processed: ../data/lip_reading/synchronized/test/M0047_03_3792846105_Android_nexus.npz(frames num: 478)
processed: ../data/lip_reading/synchronized/test/F0015_02_6107523948_Android_htc.npz(frames num: 614)
processed: ../data/lip_reading/synchronized/test/F0173_01_72901_Android_SM.npz(frames num: 241)
processed: ../data/lip_reading/synchronized/test/F0090_01_97842_Android_SM.npz(frames num: 536)
processed: ../data/lip_reading/synchronized/test/F0201_01_4059286173_Android_SM.npz(frames num: 491)
processed: ../data/lip_reading/synchronized/test/M0044_03_94518_iPhone_6s.npz(frames num: 473)
processed: ../data/lip_reading/synchronized/test/F0052_02_04279_iPhone_iphone6.npz(frames num: 320)
processed: ../data/lip_reading/synchronized/test/M0058_01_2637015489_iPhone_6s.npz(frames num: 810)
proces

processed: ../data/lip_reading/synchronized/test/M0044_01_4682107593_iPhone_Iphone5.npz(frames num: 699)
processed: ../data/lip_reading/synchronized/test/M0053_02_24183_Android_SM.npz(frames num: 260)
processed: ../data/lip_reading/synchronized/test/M0088_01_03156_Android_SM.npz(frames num: 286)
processed: ../data/lip_reading/synchronized/test/M0088_02_43276_Android_nexus.npz(frames num: 292)
processed: ../data/lip_reading/synchronized/test/M0081_03_6734829150_Android_htc.npz(frames num: 478)
processed: ../data/lip_reading/synchronized/test/F0042_02_04279_iPhone_iphone6.npz(frames num: 290)
processed: ../data/lip_reading/synchronized/test/F0169_01_49608_iPhone_6s.npz(frames num: 499)
processed: ../data/lip_reading/synchronized/test/F0069_01_70531_iPhone_6s.npz(frames num: 311)
processed: ../data/lip_reading/synchronized/test/M0060_02_97063_Android_nexus.npz(frames num: 305)
processed: ../data/lip_reading/synchronized/test/M0037_02_1392408567_iPhone_iphone6.npz(frames num: 583)
processe

processed: ../data/lip_reading/synchronized/test/F0011_03_5897106423_Android_SM_Gal_J3.npz(frames num: 676)
processed: ../data/lip_reading/synchronized/test/F0200_02_8452379610_Android_SM.npz(frames num: 632)
processed: ../data/lip_reading/synchronized/test/M0059_01_87901_iPhone_6s.npz(frames num: 539)
processed: ../data/lip_reading/synchronized/test/F0052_03_60475_iPhone_6s.npz(frames num: 176)
processed: ../data/lip_reading/synchronized/test/F0191_02_0369245817_Android_nexus.npz(frames num: 452)
processed: ../data/lip_reading/synchronized/test/F0068_02_74096_Android_SM_Gal_J3.npz(frames num: 491)
processed: ../data/lip_reading/synchronized/test/M0049_03_53970_Android_SM_Gal_J3.npz(frames num: 344)
processed: ../data/lip_reading/synchronized/test/F0029_02_04279_iPhone_iphone6.npz(frames num: 338)
processed: ../data/lip_reading/synchronized/test/M0046_01_4027851963_Android_htc.npz(frames num: 838)
processed: ../data/lip_reading/synchronized/test/M0016_02_9367254810_Android_htc.npz(fram

processed: ../data/lip_reading/synchronized/test/F0197_01_7546301298_Android_nexus.npz(frames num: 945)
processed: ../data/lip_reading/synchronized/test/F0169_02_8036592741_Android_htc.npz(frames num: 878)
processed: ../data/lip_reading/synchronized/test/M0047_02_65021_iPhone_6s.npz(frames num: 361)
processed: ../data/lip_reading/synchronized/test/M0077_03_31759_iPhone_6s.npz(frames num: 473)
processed: ../data/lip_reading/synchronized/test/F0090_01_50216_iPhone_6s.npz(frames num: 447)
processed: ../data/lip_reading/synchronized/test/M0039_02_6780524931_Android_htc.npz(frames num: 670)
processed: ../data/lip_reading/synchronized/test/M0065_03_4532091867_iPhone_iphone6.npz(frames num: 598)
processed: ../data/lip_reading/synchronized/test/M0076_03_4532091867_iPhone_iphone6.npz(frames num: 422)
processed: ../data/lip_reading/synchronized/test/F0050_03_06834_Android_SM.npz(frames num: 446)
processed: ../data/lip_reading/synchronized/test/M0062_03_4532091867_iPhone_iphone6.npz(frames num: 5

processed: ../data/lip_reading/synchronized/test/M0072_02_49586_Android_nexus.npz(frames num: 356)
processed: ../data/lip_reading/synchronized/test/M0045_03_78210_iPhone_6s.npz(frames num: 436)
processed: ../data/lip_reading/synchronized/test/F0201_01_6503279481_iPhone_6s.npz(frames num: 451)
processed: ../data/lip_reading/synchronized/test/M0059_02_17538_Android_htc.npz(frames num: 386)
processed: ../data/lip_reading/synchronized/test/M0028_02_34598_iPhone_Iphone5.npz(frames num: 326)
processed: ../data/lip_reading/synchronized/test/M0077_01_47016_iPhone_6s.npz(frames num: 427)
processed: ../data/lip_reading/synchronized/test/F0090_01_6271853904_iPhone_6s.npz(frames num: 846)
processed: ../data/lip_reading/synchronized/test/F0169_01_4923510687_Android_SM.npz(frames num: 881)
processed: ../data/lip_reading/synchronized/test/F0034_03_03582_Android_SM_Gal_J3.npz(frames num: 350)
processed: ../data/lip_reading/synchronized/test/F0021_02_2174905683_Android_htc.npz(frames num: 522)
processe

In [27]:
[tensor.name for tensor in tf.get_default_graph().as_graph_def().node]

['global_step/initial_value',
 'global_step',
 'global_step/Assign',
 'global_step/read',
 'features',
 'features_len',
 'targets',
 'targets_len',
 'learning_rate/input',
 'learning_rate',
 'is_training/input',
 'is_training',
 'cond/Switch',
 'cond/switch_t',
 'cond/switch_f',
 'cond/pred_id',
 'cond/Const',
 'cond/Const_1',
 'cond/Merge',
 'conv3d/kernel/Initializer/random_uniform/shape',
 'conv3d/kernel/Initializer/random_uniform/min',
 'conv3d/kernel/Initializer/random_uniform/max',
 'conv3d/kernel/Initializer/random_uniform/RandomUniform',
 'conv3d/kernel/Initializer/random_uniform/sub',
 'conv3d/kernel/Initializer/random_uniform/mul',
 'conv3d/kernel/Initializer/random_uniform',
 'conv3d/kernel',
 'conv3d/kernel/Assign',
 'conv3d/kernel/read',
 'conv3d/bias/Initializer/zeros',
 'conv3d/bias',
 'conv3d/bias/Assign',
 'conv3d/bias/read',
 'conv3d/convolution/Shape',
 'conv3d/convolution/dilation_rate',
 'conv3d/convolution',
 'conv3d/BiasAdd',
 'conv3d/Relu',
 'max_pooling3d/MaxPo

In [32]:
features_tensor = tf.get_default_graph().get_tensor_by_name("rnn2/concat:0")

In [33]:
features_tensor

<tf.Tensor 'rnn2/concat:0' shape=(?, ?, 512) dtype=float32>

In [36]:
for i, file in tqdm.tqdm(enumerate(files_train)):
    video_features = dnn.get_features_video(video_train[i], np.array(len(video_train[i])).reshape(1,))
    np.savez(os.path.join("../data/lip_reading/synchronized/video_features/train/", os.path.basename(file)), video_features=video_features)

8433it [38:31,  4.12it/s]


In [37]:
for i, file in tqdm.tqdm(enumerate(files_val)):
    video_features = dnn.get_features_video(video_val[i], np.array(len(video_val[i])).reshape(1,))
    np.savez(os.path.join("../data/lip_reading/synchronized/video_features/train/", os.path.basename(file)), video_features=video_features)

937it [04:14,  4.71it/s]


In [38]:
for i, file in tqdm.tqdm(enumerate(files_test)):
    video_features = dnn.get_features_video(video_test[i], np.array(len(video_test[i])).reshape(1,))
    np.savez(os.path.join("../data/lip_reading/synchronized/video_features/test/", os.path.basename(file)), video_features=video_features)

2909it [14:31,  3.67it/s]
